In [1]:
using Revise

In [2]:
using ParallelTemperingMonteCarlo

In [4]:
testensemble = NVT(10)

ms = MoveStrategy(testensemble,10,0,1)

MethodError: MethodError: no method matching MoveStrategy(::NVT, ::Int64, ::Int64, ::Int64)
Closest candidates are:
  MoveStrategy(::Any, ::Any, ::Any) at ~/.julia/packages/ParallelTemperingMonteCarlo/Fb2kY/src/MCMoves.jl:26

In [3]:
length(ms)


UndefVarError: UndefVarError: ms not defined